# Super-Convergence Learning Rate Schedule  (TensorFlow Backend) 
In this example we will implement super-convergence learning rate (LR) schedule (https://arxiv.org/pdf/1708.07120.pdf) and test it on a CIFAR10 image classification task. Super-covergence is a phenomenon where neural networks can be trained an order of magnitude faster than with standard training methods. The paper proposes a LR schedule which incorporates two parts: a LR range test to find the appropriate LR range and a cyclical LR schedule that uses the obtained information. 

In [1]:
import tempfile

import numpy as np
import tensorflow as tf
from tensorflow.python.keras import layers

import fastestimator as fe
from fastestimator.backend import get_lr
from fastestimator.dataset.data.cifair10 import load_data
from fastestimator.op.numpyop.meta import Sometimes
from fastestimator.op.numpyop.multivariate import HorizontalFlip, PadIfNeeded, RandomCrop
from fastestimator.op.numpyop.univariate import CoarseDropout, Normalize, Onehot
from fastestimator.op.tensorop.loss import CrossEntropy
from fastestimator.op.tensorop.model import ModelOp, UpdateOp
from fastestimator.trace import Trace
from fastestimator.trace.adapt import LRScheduler
from fastestimator.trace.io import BestModelSaver
from fastestimator.trace.metric import Accuracy
from fastestimator.util.util import Suppressor

In [2]:
# Parameters
epochs=24
batch_size=128
lr_epochs=100
max_train_steps_per_epoch=None
save_dir=tempfile.mkdtemp()

## Network Architecture and Data Pipeline
We will use almost the same image classification configuration of the other Apphub example: [CIFAR10 Fast](../../image_classification/cifar10_fast/cifar10_fast.ipynb) including network architecture and data pipeline. The only difference is that we use SGD optimizer instead of Adam because author of the paper specially pointed out the incompatibility between Adam optimizer and super-convergence. 

In [3]:
def residual(x, num_channel):
    x = layers.Conv2D(num_channel, 3, padding='same')(x)
    x = layers.BatchNormalization(momentum=0.8)(x)
    x = layers.LeakyReLU(alpha=0.1)(x)
    x = layers.Conv2D(num_channel, 3, padding='same')(x)
    x = layers.BatchNormalization(momentum=0.8)(x)
    x = layers.LeakyReLU(alpha=0.1)(x)
    return x


def my_model():
    # prep layers
    inp = layers.Input(shape=(32, 32, 3))
    x = layers.Conv2D(64, 3, padding='same')(inp)
    x = layers.BatchNormalization(momentum=0.8)(x)
    x = layers.LeakyReLU(alpha=0.1)(x)
    # layer1
    x = layers.Conv2D(128, 3, padding='same')(x)
    x = layers.MaxPool2D()(x)
    x = layers.BatchNormalization(momentum=0.8)(x)
    x = layers.LeakyReLU(alpha=0.1)(x)
    x = layers.Add()([x, residual(x, 128)])
    # layer2
    x = layers.Conv2D(256, 3, padding='same')(x)
    x = layers.MaxPool2D()(x)
    x = layers.BatchNormalization(momentum=0.8)(x)
    x = layers.LeakyReLU(alpha=0.1)(x)
    # layer3
    x = layers.Conv2D(512, 3, padding='same')(x)
    x = layers.MaxPool2D()(x)
    x = layers.BatchNormalization(momentum=0.8)(x)
    x = layers.LeakyReLU(alpha=0.1)(x)
    x = layers.Add()([x, residual(x, 512)])
    # layers4
    x = layers.GlobalMaxPool2D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(10)(x)
    x = layers.Activation('softmax', dtype='float32')(x)
    model = tf.keras.Model(inputs=inp, outputs=x)

    return model

# prepare dataset
train_data, test_data = load_data()
pipeline = fe.Pipeline(
    train_data=train_data,
    eval_data=test_data,
    batch_size=batch_size,
    ops=[
        Normalize(inputs="x", outputs="x", mean=(0.4914, 0.4822, 0.4465), std=(0.2471, 0.2435, 0.2616)),
        PadIfNeeded(min_height=40, min_width=40, image_in="x", image_out="x", mode="train"),
        RandomCrop(32, 32, image_in="x", image_out="x", mode="train"),
        Sometimes(HorizontalFlip(image_in="x", image_out="x", mode="train")),
        CoarseDropout(inputs="x", outputs="x", mode="train", max_holes=1),
        Onehot(inputs="y", outputs="y", mode="train", num_classes=10, label_smoothing=0.2)
    ])

# prepare network
model = fe.build(model_fn=my_model, optimizer_fn="sgd")
network = fe.Network(ops=[
    ModelOp(model=model, inputs="x", outputs="y_pred"),
    CrossEntropy(inputs=("y_pred", "y"), outputs="ce"),
    UpdateOp(model=model, loss_name="ce")
])

## LR Range Test
The preparation of the super-convergence schedule is to search the suitable LR range. The process is training the target network with a linearly increasing LR and observing the validation accuracy. Generally, the accuracy will keep increase until at some certain point when the LR get too high and start making training diverge. The very LR of that moment is the "maximum LR".

To run the test we need to implement the trace to record the maximum LR. After running the training with linear increaseing LR, we will get the maximum LR. 

<img src="./typical_lr.PNG" alt="drawing" width="400"/>
[The typical learning rate and metircs plot from https://arxiv.org/pdf/1708.07120.pdf]

In [4]:
def linear_increase(step, min_lr=0.0, max_lr=3.0, num_steps=5000):
    lr = step / num_steps * (max_lr - min_lr) + min_lr
    return lr

class GetMaxLR(Trace):
    def __init__(self, model, metric, mode):
        super().__init__(inputs=metric, outputs=None, mode=mode)
        self.model = model
        self.metric = metric
        self.best = 0
        self.max_lr = None

    def on_epoch_end(self, data):
        if data[self.metric] > self.best:
            self.best = data[self.metric]
            self.max_lr = get_lr(model=self.model)

    def get_max_lr(self):
        return self.max_lr

    def get_best(self):
        return self.best

get_max_lr_trace = GetMaxLR(model=model, metric="accuracy", mode="eval")

traces = [
    Accuracy(true_key="y", pred_key="y_pred"),
    LRScheduler(model=model, lr_fn=lambda step: linear_increase(step)),
    get_max_lr_trace
]

# prepare estimator
LR_range_test = fe.Estimator(pipeline=pipeline,
                             network=network,
                             epochs=lr_epochs,
                             traces=traces,
                             max_train_steps_per_epoch=10)

# run the LR_range_test
with Suppressor(): 
    LR_range_test.fit()

lr_max = get_max_lr_trace.get_max_lr()
print(f"The maximum LR: {lr_max}")

The maximum LR: 0.6000000238418579


## Super-Convergence LR Schedule

Once we get the maximum LR, the minimum LR can be computed by dividing it by 4 (this number is set to be 4 in the paper paragraph but fall in range of [4, 40] in its experiment section). 

The LR change has 3 phases:
1. increase LR from minimum LR to maximum LR at 0~45% of training process
2. decrase LR from maximum LR to minimum LR at 45%~90% of training process
3. decrase LR from minimum LR to 0 at 90%~100% of training process

<img src="./lr_schedule.PNG" alt="drawing" width="400"/>

In [5]:
lr_min = lr_max / 4
mid = int(epochs * 0.45 * len(train_data) / batch_size)
end = int(epochs * len(train_data) / batch_size)

def super_schedule(step):
    if step < mid:
        lr = step / mid * (lr_max - lr_min) + lr_min  # linear increase from lr_min to lr_max

    elif mid <= step < mid * 2:
        lr = lr_max - (step - mid) / mid * (lr_max - lr_min)  # linear decrease from lr_max to lr_min

    else:
        lr = max(lr_min - (step - 2 * mid) / (end - 2 * mid) * lr_min, 0)  # linear decrease from lr_min to 0

    return lr

Before we start the main training, the model needs to be reinitialized.  Therefore we re-instantiate the same network and plug the new LR scheduler in the estimator.

In [6]:
# reinitialize the model
model = fe.build(model_fn=my_model, optimizer_fn="sgd")
network = fe.Network(ops=[
    ModelOp(model=model, inputs="x", outputs="y_pred"),
    CrossEntropy(inputs=("y_pred", "y"), outputs="ce"),
    UpdateOp(model=model, loss_name="ce")
])

traces = [
    Accuracy(true_key="y", pred_key="y_pred"),
    BestModelSaver(model=model, save_dir=save_dir, metric="accuracy", save_best_mode="max"),
    LRScheduler(model=model, lr_fn=lambda step: super_schedule(step))
]

# prepare estimator 
main_train = fe.Estimator(pipeline=pipeline,
                          network=network,
                          epochs=epochs,
                          traces=traces,
                          max_train_steps_per_epoch=max_train_steps_per_epoch)

main_train.fit()

    ______           __  ______     __  _                 __            
   / ____/___ ______/ /_/ ____/____/ /_(_)___ ___  ____ _/ /_____  _____
  / /_  / __ `/ ___/ __/ __/ / ___/ __/ / __ `__ \/ __ `/ __/ __ \/ ___/
 / __/ / /_/ (__  ) /_/ /___(__  ) /_/ / / / / / / /_/ / /_/ /_/ / /    
/_/    \__,_/____/\__/_____/____/\__/_/_/ /_/ /_/\__,_/\__/\____/_/     
                                                                        

FastEstimator-Start: step: 1; logging_interval: 100; num_device: 1;
FastEstimator-Train: step: 1; ce: 5.3190966; model1_lr: 0.1501067;
FastEstimator-Train: step: 100; ce: 3.728242; model1_lr: 0.16066857; steps/sec: 33.95;
FastEstimator-Train: step: 200; ce: 3.5699914; model1_lr: 0.17133713; steps/sec: 33.24;
FastEstimator-Train: step: 300; ce: 2.1981258; model1_lr: 0.1820057; steps/sec: 33.63;
FastEstimator-Train: step: 391; epoch: 1; epoch_time: 12.44 sec;
FastEstimator-BestModelSaver: Saved model to /tmp/tmp8x744igq/model1_best_accuracy.h5
FastEstimator

FastEstimator-BestModelSaver: Saved model to /tmp/tmp8x744igq/model1_best_accuracy.h5
FastEstimator-Eval: step: 5083; epoch: 13; accuracy: 0.8298; ce: 0.61346173; max_accuracy: 0.8298; since_best_accuracy: 0;
FastEstimator-Train: step: 5100; ce: 1.1839432; model1_lr: 0.5059033; steps/sec: 32.59;
FastEstimator-Train: step: 5200; ce: 1.2401822; model1_lr: 0.49523473; steps/sec: 33.32;
FastEstimator-Train: step: 5300; ce: 1.1422856; model1_lr: 0.48456615; steps/sec: 33.7;
FastEstimator-Train: step: 5400; ce: 1.160527; model1_lr: 0.4738976; steps/sec: 33.46;
FastEstimator-Train: step: 5474; epoch: 14; epoch_time: 11.74 sec;
FastEstimator-Eval: step: 5474; epoch: 14; accuracy: 0.8124; ce: 0.6909418; max_accuracy: 0.8298; since_best_accuracy: 1;
FastEstimator-Train: step: 5500; ce: 1.1148993; model1_lr: 0.46322903; steps/sec: 32.75;
FastEstimator-Train: step: 5600; ce: 1.0829599; model1_lr: 0.45256048; steps/sec: 33.49;
FastEstimator-Train: step: 5700; ce: 1.1756632; model1_lr: 0.4418919; st

## Result Discussion
The result of it might not be super impressive when comparing with original example [CIFAR10 Fast](../../image_classification/cifar10_fast/cifar10_fast.ipynb). But please be aware that the example has its own LR schedules which is specially tuned on that configuration (plus that scheduler is also cyclical LR schedule).